In [ ]:
# from https://becominghuman.ai/part-of-speech-tagging-tutorial-with-the-keras-deep-learning-library-d7f93fa05537

In [1]:
import numpy as np
CUSTOM_SEED = 42
np.random.seed(CUSTOM_SEED)

In [3]:
import nltk

In [4]:
nltk.download('treebank')

[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\anuryadav\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\treebank.zip.


True

In [10]:
from nltk.corpus import treebank
nltk.download('universal_tagset')
sentences = treebank.tagged_sents(tagset='universal')

[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\anuryadav\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\universal_tagset.zip.


In [11]:
sentences

[[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')], [('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')], ...]

In [12]:
import random
print(random.choice(sentences))

[('The', 'DET'), ('vacation', 'NOUN'), ('packages', 'NOUN'), ('include', 'VERB'), ('hotel', 'NOUN'), ('accommodations', 'NOUN'), ('and', 'CONJ'), (',', '.'), ('in', 'ADP'), ('some', 'DET'), ('cases', 'NOUN'), (',', '.'), ('tours', 'NOUN'), ('or', 'CONJ'), ('tickets', 'NOUN'), ('to', 'PRT'), ('local', 'ADJ'), ('attractions', 'NOUN'), (',', '.'), ('but', 'CONJ'), ('not', 'ADV'), ('meals', 'NOUN'), ('.', '.')]


In [16]:
tags = set([
    tag for sentence in treebank.tagged_sents() 
    for _, tag in sentence
])
print('nb_tags: %sntags: %s' % (len(tags), tags))

train_test_cutoff = int(.8 * len(sentences))

In [18]:
training_sentences = sentences[:train_test_cutoff]

testing_sentences = sentences[train_test_cutoff:]

train_val_cutoff = int(.25 * len(training_sentences))

validation_sentences = training_sentences[:train_val_cutoff]

trainning_sentences = training_sentences[train_val_cutoff:]

In [33]:
def add_basic_features(sentence_terms, index):
    term = sentence_terms[index]
    return {
        
        'nb_terms': len(sentence_terms),
        'term': term,
        'is_last': index == len(sentence_terms) - 1,
        'is_capitalized': term[0].upper() == term[0],
        'is_all_caps': term.upper() == term,
        'is_all_lower': term.lower() == term,
        'prefix-1': term[0],
        'prefix-2': term[:2],
        'prefix-3': term[:3],
        'suffix-1': term[-1],
        'suffix-2': term[-2:],
        'suffix-3': term[-3:],
        'prev_word': '' if index == 0 else sentence_terms[index - 1],
        'next_word': '' if index == len(sentence_terms) - 1 else sentence_terms[index + 1]
        
    }

def untag(tagged_sentence):
    """ 
    Remove the tag for each tagged term.
:param tagged_sentence: a POS tagged sentence
    :type tagged_sentence: list
    :return: a list of tags
    :rtype: list of strings
    """
    return [w for w, _ in tagged_sentence]

def transform_to_dataset(tagged_sentences):
    """
    Split tagged sentences to X and y datasets and append some basic features.
    :param tagged_sentences: a list of POS tagged sentences
    :param tagged_sentences: list of list of tuples (term_i, tag_i)
    :return: 
    """
    X, y = [], []
    for pos_tags in tagged_sentences:
        for index, (term, class_) in enumerate(pos_tags):
            # Add basic NLP features for each sentence term
            X.append(add_basic_features(untag(pos_tags), index))
            y.append(class_)
    return X, y


In [34]:
X_train, y_train = transform_to_dataset(training_sentences)
X_test, y_test = transform_to_dataset(testing_sentences)
X_val, y_val = transform_to_dataset(validation_sentences)

In [39]:
from sklearn.feature_extraction import DictVectorizer
# Fit our DictVectorizer with our set of features
dict_vectorizer = DictVectorizer(sparse=False)
dict_vectorizer.fit(X_train + X_test + X_val)


DictVectorizer(dtype=<class 'numpy.float64'>, separator='=', sort=True,
        sparse=False)

In [56]:
# Convert dict features to vectors
X_train = dict_vectorizer.transform(X_train)


X_val = dict_vectorizer.transform(X_val)

len(X_val)



X_test = dict_vectorizer.transform(X_test)

MemoryError: 